In [ ]:
# Install necessary packages
!pip install yt-dlp moviepy transformers openai-whisper numpy

In [ ]:
from yt_dlp import YoutubeDL
from moviepy.editor import VideoFileClip, concatenate_videoclips
import whisper
from transformers import pipeline

In [ ]:
# Install necessary packages
!pip install yt-dlp moviepy transformers openai-whisper numpy

In [ ]:
from yt_dlp import YoutubeDL
from moviepy.editor import VideoFileClip, concatenate_videoclips
import whisper
from transformers import pipeline

In [ ]:
# Step 1: Download the YouTube video
def download_youtube_video(video_url):
    ydl_opts = {'outtmpl': 'video.mkv'}
    with YoutubeDL(ydl_opts) as ydl:
        ydl.download([video_url])
    return "video.mkv"

# Step 2: Extract audio from the video
def extract_audio(video_path):
    video_clip = VideoFileClip(video_path)
    audio_path = "audio.wav"
    video_clip.audio.write_audiofile(audio_path)
    return audio_path, video_clip

# Step 3: Transcribe the audio
def transcribe_audio(audio_path):
    model = whisper.load_model("base")
    result = model.transcribe(audio_path)
    return result["text"], result["segments"]

# Step 4: Summarize the transcript
def summarize_transcript(transcript):
    summarizer = pipeline("summarization")
    max_chunk_size = 1024  # Define the maximum chunk size

    # Split transcript into chunks
    chunks = [transcript[i:i + max_chunk_size] for i in range(0, len(transcript), max_chunk_size)]
    summarized_chunks = []

    for chunk in chunks:
        summary = summarizer(chunk, max_length=150, min_length=30, do_sample=False)
        summarized_chunks.append(summary[0]['summary_text'])

    # Combine summarized chunks
    combined_summary = ' '.join(summarized_chunks)
    return combined_summary

# Step 5: Extract key segments from video
def extract_key_segments(video_clip, segments, summary):
    keywords = summary.split()
    key_segments = []

    for segment in segments:
        for keyword in keywords:
            if keyword.lower() in segment['text'].lower():
                start = segment['start']
                end = segment['end']
                key_clip = video_clip.subclip(start, end)
                key_segments.append(key_clip)
                break

    return key_segments

# Main function
def main(video_url):
    video_path = download_youtube_video(video_url)
    audio_path, video_clip = extract_audio(video_path)
    transcript, segments = transcribe_audio(audio_path)
    summary = summarize_transcript(transcript)
    key_segments = extract_key_segments(video_clip, segments, summary)

    if key_segments:
        final_clip = concatenate_videoclips(key_segments)
        final_clip.write_videofile("summary_video.mp4", codec="libx264")
        print("Summarized video saved as summary_video.mp4")
    else:
        print("No key segments found.")

In [ ]:
if __name__ == "__main__":
    video_url = "https://youtu.be/zvkNBeoCwA0?si=nUqDXaMoUdPBf5Yt"
    main(video_url)

[youtube] Extracting URL: https://youtu.be/Z6nkEZyS9nA?si=9dG0Oh5iMTyFLYUu
[youtube] Z6nkEZyS9nA: Downloading webpage
[youtube] Z6nkEZyS9nA: Downloading ios player API JSON
[youtube] Z6nkEZyS9nA: Downloading m3u8 information
[info] Z6nkEZyS9nA: Downloading 1 format(s): 137+251
[download] video.mkv has already been downloaded
MoviePy - Writing audio in audio.wav


MoviePy - Done.


  warnings.warn("FP16 is not supported on CPU; using FP32 instead")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
  warnings.warn(



Moviepy - Building video summary_vvideo.mp4.
MoviePy - Writing audio in summary_vvideoTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video summary_vvideo.mp4



Moviepy - Done !
Moviepy - video ready summary_vvideo.mp4
Summarized video saved as summary_video.mp4


In [ ]:
from google.colab import files

# Download the video file
files.download("summary_vvideo.mp4")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>